# How to deal with complex/large Documents

In the previous notebook, we developed a solution for various types of files and data formats commonly found in organizations, and this covers big majority of the use cases. However, you will find that there are issues when dealing with questions that require answers from complex files. The complexity of these files arises from their length and the way information is distributed within them. Large documents are always a challenge for Search Engines.

One example of such complex files is Technical Specification Guides or Product Manuals, which can span hundreds of pages and contain information in the form of images, tables, forms, and more. Books are also complex due to their length and the presence of images or tables.

These files are typically in PDF format. To better handle these PDFs, we need a smarter parsing method that treats each document as a special source and processes them page by page (1 page = 1 chunk). The objective is to obtain more accurate and faster answers from our system. Fortunately, there are usually not many of these types of documents in an organization, allowing us to make exceptions and treat them differently.

If your use case is just PDFs, for example, you can just use [PyPDF library](https://pypi.org/project/pypdf/) or [Azure AI Document Intelligence SDK (former Form Recognizer)](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-3.0.0), vectorize using OpenAI API and push the content to a vector-based index. And this is problably the simplest and fastest way to go.  However if your use case entails connecting to a datalake, or Sharepoint libraries or any other document data source with thousands of documents with multiple file types and that can change dynamically, then you would want to use the Ingestion and Document Cracking and AI-Enrichment capabilities of Azure Search engine, Notebooks 1-3, and avoid a lot of painful custom code. 


In [1]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
from typing import List

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from azure.identity import ManagedIdentityCredential
import openai


from common.utils import parse_pdf, read_pdf_files, text_to_base64
from common.prompts import DOCSEARCH_PROMPT
from common.utils import CustomAzureSearchRetriever


from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
BLOB_CONTAINER_NAME = "books"
BASE_CONTAINER_URL = "https://blobstoragejed5nzg3k2jp6.blob.core.windows.net/" + BLOB_CONTAINER_NAME + "/" # YK
# go to storage account->endpoints->blob service
LOCAL_FOLDER = "./data/books/"
#YK
MODEL = "gpt-4o" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k 

os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
#os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
# Step 1: Specify the client ID of the User Managed Identity
user_managed_identity_client_id = "d30cba06-04c1-4065-a91d-8b7ce3b07b78"  # Replace with your User Managed Identity client ID

# Step 2: Fetch the access token using ManagedIdentityCredential and the client ID of the user-managed identity
credential = ManagedIdentityCredential(client_id=user_managed_identity_client_id)
token = credential.get_token("https://cognitiveservices.azure.com/.default")

# Step 3: Set the access token in the OpenAI API
openai.api_key = token.token
openai.api_type = "azure"
openai.api_base = "https://azuremlopenai.openai.azure.com/"  # Replace with your OpenAI resource's base URL
openai.api_version = "2023-06-01-preview"  # Use the correct API version


In [4]:
batch_size = 75
embedder = AzureOpenAIEmbeddings(openai_api_key=token.token,deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=batch_size, 
                                 max_retries=2, 
                                 retry_min_seconds= 60,
                                 retry_max_seconds= 70)

## 1 - Manual Document Cracking with Push to Vector-based Index

Within our demo storage account, we have a container named `books`, which holds 5 books of different lengths, languages, and complexities. Let's create a `cogsrch-index-books-vector` and load it with the pages of all these books.

We begin by downloading these books to our local machine:

In [5]:
books = ["images.pdf","azure-search.pdf"]

Let's download the files to the local `./data/` folder:

In [4]:
for book in tqdm(books):
    book_url = BASE_CONTAINER_URL + book + os.environ['BLOB_SAS_TOKEN']
    urllib.request.urlretrieve(book_url, LOCAL_FOLDER+ book)

100%|██████████| 2/2 [00:06<00:00,  3.04s/it]


In [19]:
book_pages_map = dict()
for book in books:
    print("Extracting Text from",book,"...")
    
    # Capture the start time
    start_time = time.time()
    
    # Parse the PDF
    book_path = LOCAL_FOLDER+book
    book_map = parse_pdf(file=book_path, form_recognizer=False, verbose=True)
    book_pages_map[book]= book_map
    
    # Capture the end time and Calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Parsing took: {elapsed_time:.6f} seconds")
    print(f"{book} contained {len(book_map)} pages\n")

Extracting Text from images.pdf ...
Extracting text using PyPDF
Parsing took: 0.049578 seconds
images.pdf contained 2 pages

Extracting Text from azure-search.pdf ...
Extracting text using PyPDF
Parsing took: 42.657654 seconds
azure-search.pdf contained 2094 pages



## Create Vector-based index


Now that we have the content of the book's chunks (each page of each book) in the dictionary `book_pages_map`, let's create the Vector index in our Azure Search Engine where this content is going to land

In [5]:
book_index_name = "srch-index-books-yk"

In [6]:
credential = ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")

# Obtain an access token for the Azure Cognitive Search service
token = credential.get_token("https://search.azure.com/.default")

print(f"Access token: {token.token}")

# Construct the headers with the access token
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token.token}'
    }

params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

Access token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyIsImtpZCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyJ9.eyJhdWQiOiJodHRwczovL3NlYXJjaC5henVyZS5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8xNmIzYzAxMy1kMzAwLTQ2OGQtYWM2NC03ZWRhMDgyMGI2ZDMvIiwiaWF0IjoxNzI2OTg5MDc0LCJuYmYiOjE3MjY5ODkwNzQsImV4cCI6MTcyNzA3NTc3NCwiYWlvIjoiazJCZ1lOQTJqR3AwWDljNGYrV05VdUhsdVhkbEFRPT0iLCJhcHBpZCI6ImQzMGNiYTA2LTA0YzEtNDA2NS1hOTFkLThiN2NlM2IwN2I3OCIsImFwcGlkYWNyIjoiMiIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzE2YjNjMDEzLWQzMDAtNDY4ZC1hYzY0LTdlZGEwODIwYjZkMy8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6ImNlN2VhNTVkLWFjZjktNDEwNy1hODkxLWNhMDRmNGQ0MTdiMyIsInJoIjoiMC5BVVlBRThDekZnRFRqVWFzWkg3YUNDQzIwNENqRFloZW1KaEJnYm5nV3h6Rk1WanhBQUEuIiwic3ViIjoiY2U3ZWE1NWQtYWNmOS00MTA3LWE4OTEtY2EwNGY0ZDQxN2IzIiwidGlkIjoiMTZiM2MwMTMtZDMwMC00NjhkLWFjNjQtN2VkYTA4MjBiNmQzIiwidXRpIjoiQkpOc0M3NU1DVVNfVVkyVG8tRVNBQSIsInZlciI6IjEuMCIsInhtc19pZHJlbCI6IjcgMTgiLCJ4bXNfbWlyaWQiOiIvc3Vic2NyaXB0aW9ucy9mZTM4YzM3Ni1iN


Please note the following points regarding the index:

- The ParentKey field is absent.
- The page_num field is present.

The absence of the ParentKey field is due to the utilization of a PUSH method, rather than a PULL method. This approach indicates that we are not leveraging the integrated indexing provided by the Azure AI Search engine. Instead, we are engaging in the process of parsing, performing OCR, and manually creating and pushing the content along with its vectors.

This manual parsing process involves the use of either, the pyPDF library, or the Azure Document Intelligence API. These APIs allow for the segmentation of content by page rather than by a specified number of characters, which is the method employed by the Azure AI search indexer. Consequently, this enables the inclusion of page_num as a field in our index.

REST API version 2023-10-01-Preview supports external and internal vectorization. This Notebook assumes an external vectorization strategy. This API also supports:
    
- vectorSearch algorithms, hnsw and exhaustiveKnn nearest neighbors, with parameters for indexing and scoring.
- vectorProfiles for multiple combinations of algorithm configurations.

Vector search algorithms include **exhaustive k-nearest neighbors (KNN)** and **Hierarchical Navigable Small World (HNSW)**. Exhaustive KNN performs a brute-force search that scans the entire vector space. HNSW performs an approximate nearest neighbor (ANN) search. While KNN provides exact nearest neighbor search results with high accuracy, its computational cost and poor scalability make it impractical for large datasets or real-time applications. HNSW, on the other hand, offers a highly efficient and scalable solution for nearest neighbor searches by finding approximate nearest neighbors quickly, making it more suitable for large-scale and high-dimensional data applications.


check [HERE](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index?tabs=config-2023-10-01-Preview%2Crest-2023-11-01%2Cpush%2Cportal-check-index) for the details of the vector configuration.

In [11]:
index_payload = {
    "name": book_index_name,
    "vectorSearch": {
        "algorithms": [  # We are showing here 3 types of search algorithms configurations that you can do
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-hnsw-config-2",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 8,
                     "efConstruction": 800,
                     "efSearch": 800,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-eknn-config",
                 "kind": "exhaustiveKnn",
                 "exhaustiveKnnParameters": {
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    #"apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                    "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                    "authIdentity": None
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-2",
             "algorithm": "my-hnsw-config-2",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-3",
             "algorithm": "my-eknn-config",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "page_num","type": "Edm.Int32","searchable": "false","retrievable": "true"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,
            "vectorSearchProfile": "my-vector-profile-3", # we picked profile 3 to show that this index uses eKNN vs HNSW (on prior notebooks)
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
        
    ],
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [13]:
# Uncomment to debug errors
# r.text

## Upload the Document chunks and its vectors to the Index

The following code will iterate over each chunk of each book and use the Azure Search Rest API upload method to insert each document with its corresponding vector (using OpenAI embedding model) to the index.

In [17]:
# Function to process a batch of pages
def process_batch(bookname, pages):
    try:
        contents = [page[2] for page in pages]
        chunk_vectors = embedder.embed_documents(contents)
        
        upload_payload = {"value": []}
        for i, page in enumerate(pages):
            page_num = page[0] + 1
            content = page[2]
            book_url = BASE_CONTAINER_URL + bookname
            
            payload = {
                "@search.action": "upload",
                "id": text_to_base64(bookname + str(page_num)),
                "title": f"{bookname}_page_{str(page_num)}",
                "chunk": content,
                "chunkVector": chunk_vectors[i],
                "name": bookname,
                "location": book_url,
                "page_num": page_num
            }
            upload_payload["value"].append(payload)
        
        r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name + "/docs/index",
                          data=json.dumps(upload_payload), headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed to upload batch of pages from {bookname}: {r.status_code}")
            print(r.text)
    except Exception as e:
        print(f"Exception processing batch of pages from {bookname}: {e}")
        time.sleep(10)  # Wait before retrying
        process_batch(bookname, pages)  # Retry the same batch

In [20]:
%%time
for bookname, bookmap in book_pages_map.items():
        print("Uploading chunks from", bookname)
        # Split bookmap into chunks of size chunk_size
        for i in tqdm(range(0, len(bookmap), batch_size)):
            batch = bookmap[i:i + batch_size]
            process_batch(bookname, batch)

Uploading chunks from images.pdf
Uploading chunks from azure-search.pdf
CPU times: user 17.1 s, sys: 164 ms, total: 17.3 s
Wall time: 6min 13s


100%|██████████| 28/28 [06:10<00:00, 13.23s/it]
